<a href="https://colab.research.google.com/github/PranavPutsa1006/Text-Summarizer/blob/main/Text_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install html2text
!pip install wikipedia
!pip install cloudmersive_validate_api_client

     |████████████████████████████████| 71kB 5.9MB/s 
  Created wheel for cloudmersive-validate-api-client: filename=cloudmersive_validate_api_client-3.1.1-cp36-none-any.whl size=204031 sha256=4f91ea4645ce96ca3ef44178c24c57d5e76dcb74dbf7fd98ad1eccf067cff26c
  Stored in directory: /root/.cache/pip/wheels/2e/df/a6/cd18b53b6960a223e9fd3521219414db157b67de14245445bb
Successfully built cloudmersive-validate-api-client


In [8]:
import urllib.request
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import html2text
import os
import wikipedia
import cloudmersive_validate_api_client
from cloudmersive_validate_api_client.rest import ApiException


def wiki(input_gideon):
    if 'wikipedia' in input_gideon:
        statement = input_gideon.replace("wikipedia ", "")
        try:
            results = wikipedia.summary(statement, sentences=10)
            print("According to Wikipedia, "+results)
            return results
        except:
            print("Could not find anything about "+statement)


def read_article(file_name):
    file = open(file_name, "r+")
    filedata = file.readlines()
    print(filedata)
    article = filedata[0].split(". ")
    sentences = []

    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop()

    return sentences


def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)


def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:  # ignore if both are same sentences
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name, top_n=5):
    # nltk.download("stopwords")
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences = read_article(file_name)

    # Step 2 - Generate Similarity matrix across sentences
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity matrix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    # print("Indexes of top ranked_sentence order are ", ranked_sentence)

    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - output the summarize text
    print("Summarize Text: \n", ". ".join(summarize_text))


if __name__ == "__main__":

    file = open('file.txt', 'w')
    file.write(wiki("wikipedia iron man 2"))
    file.close()
    generate_summary("file.txt", 2)
    os.remove("file.txt")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
According to Wikipedia, Iron Man is a fictional superhero appearing in American comic books published by Marvel Comics. The character was co-created by writer and editor Stan Lee, developed by scripter Larry Lieber, and designed by artists Don Heck and Jack Kirby. The character made his first appearance in Tales of Suspense #39 (cover dated March 1963), and received his own title in Iron Man #1 (May 1968). Also in 1963, the character founded the Avengers alongside Thor, Ant-Man, Wasp and the Hulk.
A wealthy American business magnate, playboy, philanthropist, inventor and ingenious scientist, Anthony Edward "Tony" Stark suffers a severe chest injury during a kidnapping. When his captors attempt to force him to build a weapon of mass destruction, he instead creates a mechanized suit of armor to save his life and escape captivity. Later, Stark develops his suit, adding weapons an